### Signing Process

A signing process example for a message-bit b = 0 using Gottesman-Chuang scheme
Let Person A (Alice) want to send a message to Person B (Bob). Hash algorithms won't be considered, so Alice has to sign every single bit of her message. Message-Bit b 
∈
{
0
,
1
}
\in \{0,1\}.

Alice chooses M pairs of private keys 
{
k
0
i
,
k
1
i
}
1
≤
i
≤
M
\{k_{0}^{i},k_{1}^{i}\}\quad 1\leq i\leq M

All the 
k
0
k_{0} keys will be used to sign the message-bit if b = 0.
All the 
k
1
k_{1} keys will be used to sign the message-bit if b = 1.
The function which maps 
k
↦
|
f
k
⟩
k\mapsto |f_{k}\rangle  is known to all parties. Alice now computes the corresponding public keys 
{
|
f
k
0
i
⟩
,
|
f
k
1
i
⟩
}
\{|f_{{k_{0}}}^{i}\rangle ,|f_{{k_{1}}}^{i}\rangle \} and gives all of them to the recipients. She can make as many copies as she needs, but has to take care, not to endanger the security 
(
n
≫
T
m
 has to hold 
)
\left(n\gg Tm{\text{ has to hold }}\right).

Her level of security limits the number of identical public keys she can create

If message-bit b = 0, she sends all her private keys 
k
0
k_{0} along with the message-bit b to Bob
message-bit b = 1, she sends all her private keys 
k
1
k_{1} along with the message-bit b to Bob
Remember: In this example Alice picks only one bit b and signs it. She has to do that for every single bit in her message

In [78]:
from qiskit import *
from qiskit_ionq_provider import IonQProvider 
from qiskit.providers.jobstatus import JobStatus
#Call provider and set token value
provider = IonQProvider(token='BFmvdArkiCbsS12r4LZf5VgYDo4HngsS')
from random import randrange

from swap_test import swaptest as swap_test

%run Quantum_Hashing_Function.ipynb

In [79]:
# given a message, produces a Quantum Digital Signature
class Signature():
    def __init__(self, message):
        self.message = message
        self.priv_keys = self.generate_priv_keys()
        self.pub_keys = self.generate_pub_keys()

    # make M pairs of private keys
    def generate_priv_keys(self):
        n = 2
        d = 3
        
        return {
            'k0': [randrange(2**n - 1) for i in range(d)],
            'k1': [randrange(2**n - 1) for i in range(d)]
        }
    
    # make M pairs of public keys (Quantum States)
    def generate_pub_keys(self):
        return {
            "f_k0":[crypto_hash(ki, self.priv_keys["k0"]) for ki in self.priv_keys["k0"]], 
            "f_k1":[crypto_hash(ki, self.priv_keys["k1"]) for ki in self.priv_keys["k1"]]
        }
    
    def get_signature(self):
        return {
            "m_bit" : self.message,
            "priv_keys" : self.priv_keys['k0'] if self.message == 0b0 else self.priv_keys['k1'],
            "pub_keys" : self.pub_keys
        }

In [83]:
class Validation():
    
    THRESHOLD_CONSTANT = .5 #???
    
    def __init__(self, signature):
        self.signature = signature.get_signature()
        self.tests = self.key_tests(self.signature)
        self.res = self.validate_sig(self.tests)
    
    def key_tests(self, sig):
        priv_keys = sig["priv_keys"]
        
        pub_keys = sig["pub_keys"]['f_k0'] if self.signature["m_bit"] == 0b1 else sig["pub_keys"]['f_k1'],
        hashed_keys = [crypto_hash(ki, priv_keys) for ki in priv_keys]
        
        print("pub_keys", pub_keys)
        print("hashed_keys", hashed_keys)
        
        # perform swap test on public and hashed private keys
        tests = [swap_test(pki, pkj) for pki, pkj in zip(priv_keys, hashed_keys)]
        
        return tests
    
    def validate_sig(self, tests):
        M = len(tests)
        threshold  = M * self.THRESHOLD_CONSTANT
        
        r = sum(tests)
        if r < threshold:
            passed = 0
        passed = 1
    
        return {
            "passed": passed,
            "r": r,
            "threshold": threshold,
            "tests": self.tests,
        }
    
    def result(self):
        return self.res

In [84]:
# Message bit
b = 0

sig = Signature(b)

sig.get_signature()

{'m_bit': 0,
 'priv_keys': [0, 1, 2],
 'pub_keys': {'f_k0': [array([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0. +0.j, 0. +0.j, 0. +0.j,
          0. +0.j]),
   array([ 5.0000000e-01+0.j,  5.0000000e-01+0.j,  5.0000000e-01+0.j,
           5.0000000e-01-0.j, -2.4492936e-16+0.j,  0.0000000e+00+0.j,
           0.0000000e+00+0.j, -1.2246468e-16+0.j]),
   array([ 5.0000000e-01+0.j,  5.0000000e-01+0.j,  5.0000000e-01+0.j,
          -5.0000000e-01+0.j, -1.2246468e-16+0.j,  0.0000000e+00+0.j,
           0.0000000e+00+0.j,  6.1232340e-17+0.j])],
  'f_k1': [array([ 5.0000000e-01+0.j,  5.0000000e-01+0.j,  5.0000000e-01+0.j,
           5.0000000e-01-0.j,  0.0000000e+00+0.j, -1.2246468e-16+0.j,
          -1.2246468e-16+0.j, -1.2246468e-16+0.j]),
   array([ 5.0000000e-01+0.j,  5.0000000e-01+0.j,  5.0000000e-01+0.j,
           5.0000000e-01-0.j,  0.0000000e+00+0.j, -1.2246468e-16+0.j,
          -1.2246468e-16+0.j, -1.2246468e-16+0.j]),
   array([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0. +0.j, 0. +0.j, 0. +0.j

In [85]:
val = Validation(sig)
val.result()

pub_keys ([array([ 5.0000000e-01+0.j,  5.0000000e-01+0.j,  5.0000000e-01+0.j,
        5.0000000e-01-0.j,  0.0000000e+00+0.j, -1.2246468e-16+0.j,
       -1.2246468e-16+0.j, -1.2246468e-16+0.j]), array([ 5.0000000e-01+0.j,  5.0000000e-01+0.j,  5.0000000e-01+0.j,
        5.0000000e-01-0.j,  0.0000000e+00+0.j, -1.2246468e-16+0.j,
       -1.2246468e-16+0.j, -1.2246468e-16+0.j]), array([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0. +0.j, 0. +0.j, 0. +0.j,
       0. +0.j])],)
hashed_keys [array([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0. +0.j, 0. +0.j, 0. +0.j,
       0. +0.j]), array([ 5.0000000e-01+0.j,  5.0000000e-01+0.j,  5.0000000e-01+0.j,
        5.0000000e-01-0.j, -2.4492936e-16+0.j,  0.0000000e+00+0.j,
        0.0000000e+00+0.j, -1.2246468e-16+0.j]), array([ 5.0000000e-01+0.j,  5.0000000e-01+0.j,  5.0000000e-01+0.j,
       -5.0000000e-01+0.j, -1.2246468e-16+0.j,  0.0000000e+00+0.j,
        0.0000000e+00+0.j,  6.1232340e-17+0.j])]


TypeError: object of type 'int' has no len()